<br>
<h1 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f5 ; color : #fe346e; text-align: center; border-radius: 100px 100px;">Great Barrier Reef EDA</h1>
<br>

<img src="https://dynaimage.cdn.cnn.com/cnn/c_fill,g_auto,w_1200,h_675,ar_16:9/https%3A%2F%2Fcdn.cnn.com%2Fcnnnext%2Fdam%2Fassets%2F211104211620-great-barrier-reef-file.jpg">

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Introduction</h1></span>

<span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">Australia's stunningly beautiful Great Barrier Reef is the world’s largest coral reef and home to 1,500 species of fish, 400 species of corals, 130 species of sharks, rays, and a massive variety of other sea life.</span>

<span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">Unfortunately, the reef is under threat, in part because of the overpopulation of one particular starfish – the coral-eating crown-of-thorns starfish (or COTS for short). Scientists, tourism operators and reef managers established a large-scale intervention program to control COTS outbreaks to ecologically sustainable levels.Australia's stunningly beautiful Great Barrier Reef is the world’s largest coral reef and home to 1,500 species of fish, 400 species of corals, 130 species of sharks, rays, and a massive variety of other sea life.</span>

<span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">Unfortunately, the reef is under threat, in part because of the overpopulation of one particular starfish – the coral-eating crown-of-thorns starfish (or COTS for short). Scientists, tourism operators and reef managers established a large-scale intervention program to control COTS outbreaks to ecologically sustainable levels.</span>
<hr>
<span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">In this notebook we will visualize the images of the coral reefs along with the bounding boxes present in the dataset. They look beautiful!</span>

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Install Required Libraries</h1></span>

In [ ]:
!pip install --upgrade wandb

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Import Required Libraries 📚</h1></span>

In [ ]:
import os
import cv2
from PIL import Image

import numpy as np
import pandas as pd

from tqdm import tqdm

<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

<span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;"> Weights & Biases (W&B) is a set of machine learning tools that helps you build better models faster. <strong>Kaggle competitions require fast-paced model development and evaluation</strong>. There are a lot of components: exploring the training data, training different models, combining trained models in different combinations (ensembling), and so on.</span>

> <span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">⏳ Lots of components = Lots of places to go wrong = Lots of time spent debugging</span>

<span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">Why use W&B tables?</span>

* <span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">Well suited for Quick EDA!<br></span>
* <span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">Let's you see the entire data<br></span>
* <span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">Filter, sort and group data which can help answer some fundamental questions.<br></span>
* <span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">Well suited to visualize model predictions and compare models<br></span>

<span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">To learn more about Weights and Biases Tables check out this <strong><a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/datasets-predictions/W%26B_Tables_Quickstart.ipynb">colab notebook</a></strong>.</span>

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

# <h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Read the Data 📖</h1>

<span style="color: #000508; font-family: Segoe UI; font-size: 1.3em; font-weight: 300;">Data Structure</span>
<blockquote>
    <ul>
        <li><code>video_id</code>: ID number of the video the image was part of</li>
        <li><code>video_frame</code>: The frame number of the image within the video. Gaps might be present between frame numbers</li>
        <li><code>sequence</code>: ID of a gap-free subset of a given video</li>
        <li><code>sequence_frame</code>: The frame number within a given sequence</li>
        <li><code>annotations</code>: The bounding boxes of any starfish detections in string format. Contains <code>x_min</code>, <code>y_min</code>, <code>width</code> and <code>height</code> of the bounding box</li>
    </ul>

In [ ]:
df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
df.head()

# <h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Processing</h1>

In [ ]:
# Convert String to List
df['annotations'] = df['annotations'].apply(eval)

# Get the number of bounding boxes for each image
df['num_bboxes'] = df['annotations'].apply(lambda x: len(x))

In [ ]:
df.groupby('video_id').count()

<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">We will visualize only the images with bounding boxes</span>

In [ ]:
df_with_boxes = df[df.num_bboxes != 0].reset_index(drop=True)
df_with_boxes.shape

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Visualizations</h1></span>

In [ ]:
run = wandb.init(project='GreatBarrier',
                 job_type='Visualization',
                 name='Image Visualization',
                 anonymous='must')

In [ ]:
preview_table = wandb.Table(columns=['Video Id', 'Image', 'Video Frame', 'Num Boxes'])

for i in tqdm(range(len(df_with_boxes))):
    row = df_with_boxes.loc[i]
    img = Image.open(f'../input/tensorflow-great-barrier-reef/train_images/video_{row.video_id}/{row.video_frame}.jpg')
    bboxes_list = []
    for annot in row.annotations:
        bbox = {
            "position": {
                "minX": annot['x'],
                "maxX": annot['x'] + annot['width'],
                "minY": annot['y'],
                "maxY": annot['y'] + annot['height']
            },
            "class_id": 1,
            "box_caption": "starfish",
            "domain": "pixel"
        }
        bboxes_list.append(bbox)
        
    image = wandb.Image(img,
                        boxes = {
                            "ground_truth": {
                                "box_data": bboxes_list,
                                "class_labels" : {1: 'starfish'}
                            }
                        },
                        # Add extra dummy class to get red bounding boxes
                        classes = [{"id": 0, "name": "none"}, {"id": 1, "name": "starfish"}]
                    )
    preview_table.add_data(row.video_id,
                           image,
                           row.video_frame,
                           len(bboxes_list))

wandb.log({'Visualization': preview_table})
run.finish()

<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;"><a href="https://wandb.ai/dchanda/GreatBarrier/runs/y2nv1agh">View the Complete Table Here ⮕</a></span>

![](https://i.imgur.com/gjcGnD7.gif)

<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Work in Progress 🚧</span>

![Upvote!](https://img.shields.io/badge/Upvote-If%20you%20like%20my%20work-07b3c8?style=for-the-badge&logo=kaggle)